# Avocado analysis

## Import Data
Content This data was downloaded from the Hass Avocado Board website in May of 2018 & compiled into a single CSV. Here’s how the Hass Avocado Board describes the data on their website:

The table below represents weekly 2018 retail scan data for National retail volume (units) and price. 
Retail scan data comes directly from retailers’ cash registers based on actual retail sales of Hass avocados. 
Starting in 2013, the table below reflects an expanded, multi-outlet retail data set. 

Multi-outlet reporting includes an aggregation of the following channels: grocery, mass, club, drug, dollar and military. 

The Average Price (of avocados) in the table reflects a per unit (per avocado) cost, even when multiple units (avocados) are sold in bags. The Product Lookup codes (PLU’s) in the table are only for Hass avocados. Other varieties of avocados (e.g. greenskins) are not included in this table.

Some relevant columns in the dataset:

Date - The date of the observation 
AveragePrice - the average price of a single avocado 
type - conventional or organic 
year - the year 
Region - the city or region of the observation 
Total Volume - Total number of avocados sold 
4046 - Total number of avocados with PLU 4046 sold 
4225 - Total number of avocados with PLU 4225 sold 
4770 - Total number of avocados with PLU 4770 sold

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
%matplotlib inline
import pandas_profiling

In [55]:
import plotly.offline as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)
from plotly import tools

In [56]:
import warnings
#warnings.filterwarnings("ignore")
#warnings.filterwarnings("ignore",category=DeprecationWarning)

In [98]:
df = pd.read_csv("https://raw.githubusercontent.com/gloria08/avocado_project/master/avocado.csv")

## Data Checking and Cleaning
There are 14 variables and 18249 rows in data.

In [99]:
df.shape

(18249, 14)

In [100]:
df.columns

Index(['Unnamed: 0', 'Date', 'AveragePrice', 'Total Volume', '4046', '4225',
       '4770', 'Total Bags', 'Small Bags', 'Large Bags', 'XLarge Bags', 'type',
       'year', 'region'],
      dtype='object')

In [101]:
df.head()

,Unnamed: 0,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,0,2015-12-27,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,conventional,2015,Albany
1,1,2015-12-20,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,conventional,2015,Albany
2,2,2015-12-13,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,conventional,2015,Albany
3,3,2015-12-06,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,conventional,2015,Albany
4,4,2015-11-29,1.28,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,conventional,2015,Albany


In [102]:
 df.sample(5)

,Unnamed: 0,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
3845,49,2016-01-17,0.97,171404.92,5231.37,105606.63,12443.2,48123.72,26459.05,19618.46,2046.21,conventional,2016,Indianapolis
10565,35,2015-04-26,1.56,2747.19,353.56,83.28,0.0,2310.35,2310.35,0.00,0.00,organic,2015,NewOrleansMobile
12710,49,2016-01-17,1.53,2099.77,1295.76,471.18,0.0,332.83,326.67,6.16,0.00,organic,2016,GrandRapids
12538,33,2016-05-08,1.04,26837.25,8147.01,601.13,0.0,18089.11,12168.76,5920.35,0.00,organic,2016,DallasFtWorth
12019,34,2016-05-01,1.25,11049.83,2003.78,4136.67,0.0,4909.38,897.94,4011.44,0.00,organic,2016,Atlanta


In [103]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18249 entries, 0 to 18248
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    18249 non-null  int64  
 1   Date          18249 non-null  object 
 2   AveragePrice  18249 non-null  float64
 3   Total Volume  18249 non-null  float64
 4   4046          18249 non-null  float64
 5   4225          18249 non-null  float64
 6   4770          18249 non-null  float64
 7   Total Bags    18249 non-null  float64
 8   Small Bags    18249 non-null  float64
 9   Large Bags    18249 non-null  float64
 10  XLarge Bags   18249 non-null  float64
 11  type          18249 non-null  object 
 12  year          18249 non-null  int64  
 13  region        18249 non-null  object 
dtypes: float64(9), int64(2), object(3)
memory usage: 1.9+ MB


In [104]:
df.describe()

,Unnamed: 0,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,year
count,18249.000000,18249.000000,1.824900e+04,1.824900e+04,1.824900e+04,1.824900e+04,1.824900e+04,1.824900e+04,1.824900e+04,18249.000000,18249.000000
mean,24.232232,1.405978,8.506440e+05,2.930084e+05,2.951546e+05,2.283974e+04,2.396392e+05,1.821947e+05,5.433809e+04,3106.426507,2016.147899
std,15.481045,0.402677,3.453545e+06,1.264989e+06,1.204120e+06,1.074641e+05,9.862424e+05,7.461785e+05,2.439660e+05,17692.894652,0.939938
min,0.000000,0.440000,8.456000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,2015.000000
25%,10.000000,1.100000,1.083858e+04,8.540700e+02,3.008780e+03,0.000000e+00,5.088640e+03,2.849420e+03,1.274700e+02,0.000000,2015.000000
50%,24.000000,1.370000,1.073768e+05,8.645300e+03,2.906102e+04,1.849900e+02,3.974383e+04,2.636282e+04,2.647710e+03,0.000000,2016.000000
75%,38.000000,1.660000,4.329623e+05,1.110202e+05,1.502069e+05,6.243420e+03,1.107834e+05,8.333767e+04,2.202925e+04,132.500000,2017.000000
max,52.000000,3.250000,6.250565e+07,2.274362e+07,2.047057e+07,2.546439e+06,1.937313e+07,1.338459e+07,5.719097e+06,551693.650000,2018.000000


In [105]:
df.isnull().sum()

Unnamed: 0      0
Date            0
AveragePrice    0
Total Volume    0
4046            0
4225            0
4770            0
Total Bags      0
Small Bags      0
Large Bags      0
XLarge Bags     0
type            0
year            0
region          0
dtype: int64

no missing values

In [106]:
df.drop(['year','Unnamed: 0'],axis=1,inplace=True)

After checked the class of each columns, I dropped year column as it is duplicate. We can get it from Date column. Then I decided to convert the Date column from string to date. I also created a new column called month from Date columns in case for monthly analysis. Finally, I sorted the data by Date by ascending order.

In [107]:
df['Date']=pd.to_datetime(df['Date'])
df['Year']=df['Date'].apply(lambda x:x.year)
df['Month']=df['Date'].apply(lambda x:x.month)

In [108]:
df.head()

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,region,Year,Month
0,2015-12-27,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,conventional,Albany,2015,12
1,2015-12-20,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,conventional,Albany,2015,12
2,2015-12-13,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,conventional,Albany,2015,12
3,2015-12-06,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,conventional,Albany,2015,12
4,2015-11-29,1.28,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,conventional,Albany,2015,11


In [109]:
order = [0,-2,-1,11,10, 1,2,3,4,5,6,7,8,9] # setting column's order
df = df[[df.columns[i] for i in order]]
df.head()

,Date,Year,Month,region,type,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags
0,2015-12-27,2015,12,Albany,conventional,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0
1,2015-12-20,2015,12,Albany,conventional,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0
2,2015-12-13,2015,12,Albany,conventional,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0
3,2015-12-06,2015,12,Albany,conventional,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0
4,2015-11-29,2015,11,Albany,conventional,1.28,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0


In [113]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18249 entries, 0 to 18248
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Date          18249 non-null  datetime64[ns]
 1   Year          18249 non-null  int64         
 2   Month         18249 non-null  int64         
 3   region        18249 non-null  object        
 4   type          18249 non-null  object        
 5   AveragePrice  18249 non-null  float64       
 6   Total Volume  18249 non-null  float64       
 7   4046          18249 non-null  float64       
 8   4225          18249 non-null  float64       
 9   4770          18249 non-null  float64       
 10  Total Bags    18249 non-null  float64       
 11  Small Bags    18249 non-null  float64       
 12  Large Bags    18249 non-null  float64       
 13  XLarge Bags   18249 non-null  float64       
dtypes: datetime64[ns](1), float64(9), int64(2), object(2)
memory usage: 1.9+ MB


In [114]:
df['Year'].value_counts()

2017    5722
2016    5616
2015    5615
2018    1296
Name: Year, dtype: int64

In [115]:
df['Month'].value_counts()

1     1944
3     1836
2     1728
10    1512
7     1512
5     1512
11    1404
8     1404
4     1404
12    1403
9     1296
6     1294
Name: Month, dtype: int64

In [116]:
df['type'].value_counts()

conventional    9126
organic         9123
Name: type, dtype: int64

In [121]:
pd.crosstab(df['Year'], df['type'], dropna=False)

type,conventional,organic
Year,,
2015,2808,2807
2016,2808,2808
2017,2862,2860
2018,648,648


In [122]:
pd.crosstab(df['Year'], df['Month'], dropna=False)

Month,1,2,3,4,5,6,7,8,9,10,11,12
Year,,,,,,,,,,,,
2015,432,432,540,432,540,432,432,540,432,432,540,431
2016,540,432,432,432,540,432,540,432,432,540,432,432
2017,540,432,432,540,432,430,540,432,432,540,432,540
2018,432,432,432,0,0,0,0,0,0,0,0,0


In [125]:
pd.crosstab([df['Year'],df['Month']],df['type'])

type        conventional  organic
Year Month                       
2015 1               216      216
     2               216      216
     3               270      270
     4               216      216
     5               270      270
     6               216      216
     7               216      216
     8               270      270
     9               216      216
     10              216      216
     11              270      270
     12              216      215
2016 1               270      270
     2               216      216
     3               216      216
     4               216      216
     5               270      270
     6               216      216
     7               270      270
     8               216      216
     9               216      216
     10              270      270
     11              216      216
     12              216      216
2017 1               270      270
     2               216      216
     3               216      216
     4               270      270
     5               216      216
     6               216      214
     7               270      270
     8               216      216
     9               216      216
     10              270      270
     11              216      216
     12              270      270
2018 1               216      216
     2               216      216
     3               216      216

In [128]:
pd.crosstab(df['region'],df['Year'])

Year,2015,2016,2017,2018
region,,,,
Albany,104,104,106,24
Atlanta,104,104,106,24
BaltimoreWashington,104,104,106,24
Boise,104,104,106,24
Boston,104,104,106,24
BuffaloRochester,104,104,106,24
California,104,104,106,24
Charlotte,104,104,106,24
Chicago,104,104,106,24


In [130]:
df.region.unique()

array(['Albany', 'Atlanta', 'BaltimoreWashington', 'Boise', 'Boston',
       'BuffaloRochester', 'California', 'Charlotte', 'Chicago',
       'CincinnatiDayton', 'Columbus', 'DallasFtWorth', 'Denver',
       'Detroit', 'GrandRapids', 'GreatLakes', 'HarrisburgScranton',
       'HartfordSpringfield', 'Houston', 'Indianapolis', 'Jacksonville',
       'LasVegas', 'LosAngeles', 'Louisville', 'MiamiFtLauderdale',
       'Midsouth', 'Nashville', 'NewOrleansMobile', 'NewYork',
       'Northeast', 'NorthernNewEngland', 'Orlando', 'Philadelphia',
       'PhoenixTucson', 'Pittsburgh', 'Plains', 'Portland',
       'RaleighGreensboro', 'RichmondNorfolk', 'Roanoke', 'Sacramento',
       'SanDiego', 'SanFrancisco', 'Seattle', 'SouthCarolina',
       'SouthCentral', 'Southeast', 'Spokane', 'StLouis', 'Syracuse',
       'Tampa', 'TotalUS', 'West', 'WestTexNewMexico'], dtype=object)